In [0]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import keras
import cv2

from keras.models import Sequential,Model
from keras.layers import Input,Dense, Activation, Reshape, Convolution2D, Flatten, MaxPooling2D, Dropout
from keras.utils import np_utils

from skimage.feature import hog
from skimage import data, exposure
import pickle

Using TensorFlow backend.


In [0]:
!wget https://www.floydhub.com/api/v1/resources/tJ4C72SQps88GWr3hr88uU/X.pickle?content=true&rename=xpickle
!mv X.pickle\?content\=true X.pickle

--2018-12-18 13:32:43--  https://www.floydhub.com/api/v1/resources/tJ4C72SQps88GWr3hr88uU/X.pickle?content=true
Resolving www.floydhub.com (www.floydhub.com)... 104.25.128.32, 104.25.129.32, 2606:4700:20::6819:8120, ...
Connecting to www.floydhub.com (www.floydhub.com)|104.25.128.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘X.pickle?content=true’

X.pickle?content=tr     [   <=>              ] 359.07M  12.0MB/s    in 27s     

2018-12-18 13:33:11 (13.2 MB/s) - ‘X.pickle?content=true’ saved [376515380]



In [0]:
!wget https://www.floydhub.com/api/v1/resources/xRu8AHanSHPfwpN2AAsdjb/y.pickle?content=true&rename=ypickle
!mv y.pickle\?content\=true y.pickle

--2018-12-18 13:33:17--  https://www.floydhub.com/api/v1/resources/xRu8AHanSHPfwpN2AAsdjb/y.pickle?content=true
Resolving www.floydhub.com (www.floydhub.com)... 104.25.128.32, 104.25.129.32, 2606:4700:20::6819:8120, ...
Connecting to www.floydhub.com (www.floydhub.com)|104.25.128.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘y.pickle?content=true’

y.pickle?content=tr     [    <=>             ]   7.78M  12.1MB/s    in 0.6s    

2018-12-18 13:33:18 (12.1 MB/s) - ‘y.pickle?content=true’ saved [8154588]



In [0]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [0]:
y[y == '+'] = 10
y[y=='-'] = 11
y[y=='times'] = 12
y[y=='forward_slash'] = 13


In [0]:
y = np_utils.to_categorical(y)

In [0]:
print(X.shape,y.shape)

(156816, 2401) (156816, 14)


In [0]:
X = X[:].reshape((X.shape[0],49,49,1))

In [0]:
 from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,shuffle=True,random_state=42)

In [0]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(109771, 49, 49, 1) (47045, 49, 49, 1)
(109771, 14) (47045, 14)


In [0]:
inputs = Input(shape=(49,49,1))
conv_1 = Convolution2D(128, (4,4),strides=(2, 2),activation='relu')(inputs)
Drop_1 = Dropout(0.2)(conv_1)
conv_1 = Convolution2D(64, (3,3),strides=(2, 2),activation='relu')(Drop_1)
conv_1 = Convolution2D(32, (2,2),strides=(1, 1),activation='relu')(conv_1)
Drop_2 = Dropout(0.25)(conv_1)
max_pool = MaxPooling2D(pool_size=(2,2))(Drop_2)
flat = Flatten()(max_pool)
dense = Dense(512,activation='relu')(flat)
drop_3 = Dropout(0.2)(dense)
dense = Dense(100,activation='relu')(drop_3)
dense = Dense(14,activation='softmax')(dense)

In [0]:
conv_model = Model(inputs=inputs, outputs=dense)
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 49, 49, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 23, 128)       2176      
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        73792     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 32)        8224      
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 10, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
__________

In [0]:
encoder = Model(inputs = inputs,outputs= flat)
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 49, 49, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 23, 128)       2176      
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        73792     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 32)        8224      
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 10, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
__________

In [0]:
# decoder = Model(inputs = Input(shape=(800,)),outputs = dense)
# decoder.summary()

In [0]:
conv_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
from tensorflow.keras.callbacks import TensorBoard
import time
import tensorflow as tf

In [0]:
NAME = 'CROHME-digits-svm-20e-{}'.format(int(time.time()))

In [0]:
!ls -l --block-size=M

total 368M
-rw-r--r-- 1 root root   1M Oct 31 16:15 crohme-half-model-features--cnn-10e.h5
-rw-r--r-- 1 root root   1M Oct 31 16:15 crohme-half-model-features--cnn-20e.h5
drwxr-xr-x 2 root root   1M Oct 31 13:54 data
drwxr-xr-x 3 root root   1M Oct 31 16:08 logs
drwxr-xr-x 2 root root   1M Oct 29 16:42 sample_data
-rw-r--r-- 1 root root   1M Oct 31 16:19 svm_model_features-new-20e.sav
-rw-r--r-- 1 root root 360M Oct 31 16:07 X.pickle
-rw-r--r-- 1 root root   8M Oct 31 16:07 y.pickle


In [0]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [0]:
hist = conv_model.fit(X_train, y_train,
         epochs=20,
         shuffle=True,
         batch_size=512,
         validation_data=(X_test,y_test),callbacks = [tensorboard])

Train on 109771 samples, validate on 47045 samples
Epoch 1/20
109771/109771 [==============================] - 25s 228us/step - loss: 0.2919 - acc: 0.9249 - val_loss: 0.0409 - val_acc: 0.9897
Epoch 2/20
109771/109771 [==============================] - 22s 196us/step - loss: 0.0471 - acc: 0.9868 - val_loss: 0.0260 - val_acc: 0.9932
Epoch 3/20
109771/109771 [==============================] - 21s 195us/step - loss: 0.0289 - acc: 0.9921 - val_loss: 0.0187 - val_acc: 0.9956
Epoch 4/20
109771/109771 [==============================] - 21s 195us/step - loss: 0.0229 - acc: 0.9937 - val_loss: 0.0154 - val_acc: 0.9968
Epoch 5/20
109771/109771 [==============================] - 21s 195us/step - loss: 0.0183 - acc: 0.9947 - val_loss: 0.0154 - val_acc: 0.9958
Epoch 6/20
109771/109771 [==============================] - 21s 194us/step - loss: 0.0171 - acc: 0.9952 - val_loss: 0.0119 - val_acc: 0.9969
Epoch 7/20
109771/109771 [==============================] - 21s 195us/step - loss: 0.0141 - acc: 0.9963

In [0]:
import h5py
file_name = 'crohme-half-model-features--cnn-20e.h5'
encoder.save(file_name)  # creates a HDF5 file 'my_model.h5'
f = h5py.File(file_name, 'r+')
# del f['optimizer_weights']
f.close()

In [0]:
predictions = encoder.predict(X)
predictions.shape

(156816, 800)

In [0]:
# pickle.dump( abc, open( "features_cnn.pickle", "wb" ) )

In [0]:

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [0]:
y[y == '+'] = 10
y[y=='-'] = 11
y[y=='times'] = 12
y[y=='forward_slash'] = 13


In [0]:
X_train,X_test,y_train,y_test = train_test_split(predictions,y,test_size=0.3,shuffle=True,random_state=42)

In [0]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
clf = LinearSVC(random_state=0, tol=1e-5)

In [0]:
print(X_train.shape,y_train.shape)

(109771, 800) (109771,)


In [0]:
print(X_test.shape,y_test.shape)

(47045, 800) (47045,)


In [0]:
# clf.fit(predictions, y)
clf.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

In [0]:
from random import *
 
randNum =  randint(0, y_test.shape[0]) 
print(clf.predict(X_test[randNum].reshape(1,800))[0] == y_test[randNum])

True


'10'

In [0]:
clf.score(X_test,y_test)

0.9991710064831544

In [0]:
filename = 'svm_model_features-new-20e.sav'
pickle.dump(clf, open(filename, 'wb'))

In [0]:
# !curl --upload-file ./features_cnn.pickle https://transfer.sh/features 

In [0]:
# !wget https://transfer.sh/GTlLG/features

In [0]:
# !pip install -U floyd-cli

In [0]:
!mkdir features_cnn

In [0]:
!cp features_cnn.pickle features_cnn/

In [0]:
!floyd login -k 83b0349114365695ebbba7f81566cbe8

In [0]:
!rm -rf features_cnn

In [0]:
!floyd data init features_cnn_crohme

In [0]:
!floyd data upload

In [0]:
!mkdir pickles

In [0]:
!mv features_cnn.pickle pickles/features_cnn.pickle

In [0]:
!cd pickles && floyd data init features_cnn_crohme

In [0]:
!cd pickles && floyd data upload